# 03 - Trends + Visualizations

This notebook loads the monthly rollups and generates clean seaborn charts:

- Total month-to-month revenue (all facilities combined)
- Month-to-month revenue split by Encounter Facility

It also marks **July 2023** with a vertical dashed line (when I took over as coordinator).

In [1]:
import sys
from pathlib import Path
import pandas as pd

ROOT = Path.cwd().resolve()
if ROOT.name == "notebooks":
    ROOT = ROOT.parent

if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from src.viz import plot_total_revenue, plot_revenue_by_facility  # noqa: E402

## Load rollups

In [2]:
monthly_total = pd.read_csv(ROOT / "outputs" / "monthly_total.csv")
monthly_by_facility = pd.read_csv(ROOT / "outputs" / "monthly_by_facility.csv")

monthly_total["month"] = pd.to_datetime(monthly_total["month"])
monthly_by_facility["month"] = pd.to_datetime(monthly_by_facility["month"])

monthly_total.head()

,month,encounters,client_hours,total_units,revenue
0,2022-10-01,246,179.383333,516,16315.5
1,2022-11-01,319,191.250000,550,19085.5
2,2022-12-01,330,137.816667,420,17170.5
3,2023-01-01,350,121.033333,374,17243.0
4,2023-02-01,305,118.066667,334,15043.5


## Generate charts

In [ ]:
plot_total_revenue(
    monthly_total,
    out_dir=ROOT / "outputs",
    takeover_date="2023-08-01",
)

plot_revenue_by_facility(
    monthly_by_facility,
    out_dir=ROOT / "outputs",
    takeover_date="2023-08-01",
)

print("Saved charts to outputs/")

Saved charts to outputs/


## Quick sanity checks

In [4]:
print("Date range (total):", monthly_total["month"].min(), "to", monthly_total["month"].max())
print("Facilities:", sorted(monthly_by_facility["facility"].dropna().unique().tolist()))

Date range (total): 2022-10-01 00:00:00 to 2026-01-01 00:00:00
Facilities: ["Men's Safe Harbor", 'Outpatient', "Women's Safe Harbor"]


## Export a simple pre vs post July 2023 summary

This gives a quick before/after comparison showing how the program changed after I took over.

In [ ]:
takeover = pd.to_datetime("2023-08-01")

pre = monthly_total[monthly_total["month"] < takeover]
post = monthly_total[monthly_total["month"] >= takeover]

summary = pd.DataFrame(
    {
        "period": ["pre_takeover", "post_takeover"],
        "months": [len(pre), len(post)],
        "avg_monthly_revenue": [pre["revenue"].mean(), post["revenue"].mean()],
        "avg_monthly_client_hours": [pre["client_hours"].mean(), post["client_hours"].mean()],
        "avg_monthly_encounters": [pre["encounters"].mean(), post["encounters"].mean()],
    }
)

summary

,period,months,avg_monthly_revenue,avg_monthly_client_hours,avg_monthly_encounters
0,pre_takeover,9,19478.166667,173.737037,355.111111
1,post_takeover,31,29665.741935,321.660753,294.548387


In [6]:
# percent change (post vs pre)
pct_change = pd.DataFrame(
    {
        "metric": ["avg_monthly_revenue", "avg_monthly_client_hours", "avg_monthly_encounters"],
        "percent_change": [
            (summary.loc[1, "avg_monthly_revenue"] / summary.loc[0, "avg_monthly_revenue"] - 1) * 100,
            (summary.loc[1, "avg_monthly_client_hours"] / summary.loc[0, "avg_monthly_client_hours"] - 1) * 100,
            (summary.loc[1, "avg_monthly_encounters"] / summary.loc[0, "avg_monthly_encounters"] - 1) * 100,
        ],
    }
)

pct_change

,metric,percent_change
0,avg_monthly_revenue,52.302537
1,avg_monthly_client_hours,85.142304
2,avg_monthly_encounters,-17.054584


In [7]:
# Save summary tables
OUT_SUMMARY = ROOT / "outputs" / "takeover_summary.csv"
OUT_PCT = ROOT / "outputs" / "takeover_pct_change.csv"

summary.to_csv(OUT_SUMMARY, index=False)
pct_change.to_csv(OUT_PCT, index=False)

print("Wrote:", OUT_SUMMARY)
print("Wrote:", OUT_PCT)

Wrote: /mnt/c/Users/glopp/Python Projects/multi-year-revenue-trends/outputs/takeover_summary.csv
Wrote: /mnt/c/Users/glopp/Python Projects/multi-year-revenue-trends/outputs/takeover_pct_change.csv
